# Constraint statistics


## Preliminaries
### Import modules

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import fdrcorrection as fdr
from scipy import stats as _stats

sns.set_context("talk")

### Download data

In [2]:
%%bash
dx download -f -o ../data/ data/supplementary_dataset_11_full_constraint_metrics.tsv

Error: path "../data/supplementary_dataset_11_full_constraint_metrics.tsv"
already exists but -f/--overwrite was not set


CalledProcessError: Command 'b'dx download -o ../data/ data/supplementary_dataset_11_full_constraint_metrics.tsv\n'' returned non-zero exit status 1.

## Load the data

In [ ]:
df = pd.read_csv("../outputs/expected_variants_all_regions_no_cpg.tsv", sep="\t")

# Print summary data
_ = (df.region.value_counts() / 3).astype(int)
print(f"Number of distinct regions:\n{_}")  # 3 consequences per region (syn/mis/non)

## Get constraint Z scores

In [ ]:
def per_row_ztest(row, statistic="z"):
    if statistic == "z":
        i = 0
    elif statistic == "p":
        i = 1

    stat = proportions_ztest(
        count=row["n_obs"],
        nobs=row["n_pos"],
        value=row["prop_exp"],
        alternative="smaller",
        prop_var=row["prop_exp"],
    )[i]

    return stat

In [ ]:
%%capture
df["z"] = df.apply(per_row_ztest, statistic="z", axis=1)
df["p"] = df.apply(per_row_ztest, statistic="p", axis=1)

# Print summary data
_ = df.groupby(["region", "csq"])["z"].count()
print(f"Constraint statistics by region and consequence:\n{_}")

## Get FDR-adjusted P-values
For nonsense variants only. Calculate separately for whole-transcripts and constrained regions

In [ ]:
def fdr_adjustment(df, regions=["transcript"], csq="nonsense"):
    """Get FDR-adjusted P-values for a given region and variant consequence."""
    # Mask regions and consequences
    m1 = df.region.isin(regions)
    m2 = df.csq == csq

    # Filter the dataframe and drop cases without a P-value
    p = df.loc[m1 & m2, ["region", "p"]].dropna().copy()

    # FDR adjustment
    p["fdr_p"] = fdr(pvals=p["p"])[1]

    return p

In [ ]:
# FDR adjustment is done separately for transcripts and NMD regions
r1 = ["transcript"]
r2 = ["distal_nmd", "nmd_target", "long_exon"]

fdr_p = pd.concat([fdr_adjustment(df, regions=r) for r in [r1, r2]])

# Join FDR-adjusted p-values to the original dataframe
df = df.join(fdr_p["fdr_p"])

## Merge with gnomAD constraint data

In [ ]:
gnomad = pd.read_csv(
    "../data/supplementary_dataset_11_full_constraint_metrics.tsv",
    sep="\t",
    usecols=["transcript", "pLI", "oe_lof_upper"],
).rename(columns={"transcript": "enst", "pLI": "pli", "oe_lof_upper": "loeuf"})

df = df.merge(gnomad, how="left")

## Save to output

In [ ]:
df.to_csv("../outputs/expected_variants_all_regions_no_cpg_stats.tsv", sep="\t")